In [1]:
import os
import tensorflow as tf
import cv2
import ntpath
import numpy as np

2023-06-22 16:42:18.475874: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
weights_path = "/Users/andreacivitarese/PycharmProjects/CVandDL/resnet50/weights/weights_resnet_continued_notfrozen_15_lr_0.001_dropout_0.5_batch_size_32_pooling_max.h5"

image_to_predict_path = "/Users/andreacivitarese/PycharmProjects/CVandDL/dataset_classificazione/women/249_TOD'S_leather.jpg"
input_shape = (224, 224)

possible_materials = {"goat-lether":0, "raffia":1, "polyurethane":2, "calf-leather":3, "leather":4, "fabric":6, "lamb-leather":5, "polyester":9, "cow-leather":7, "canvas":8}

In [20]:
img = cv2.imread(image_to_predict_path)
img = cv2.resize(img, input_shape)
img = tf.keras.applications.resnet50.preprocess_input(img)
img = np.asarray([img])

head, image_name = ntpath.split(image_to_predict_path)
material = image_name.split('_')[2].split('.')[0]
label = np.zeros(shape=10, dtype=float)
index = possible_materials[material]
label[index] = 1

In [21]:
resnet = tf.keras.applications.resnet50.ResNet50(
        input_shape=(224, 224, 3),  # Making the image into 3 Channel
        weights='imagenet',
        include_top=False,
        pooling='max'
    )
for layer in resnet.layers[:-15]:
    layer.trainable = False
model = tf.keras.Sequential()
model.add(resnet)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

model.compile(loss="categorical_crossentropy", metrics="Recall",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9))

model.load_weights(weights_path)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_4 (Flatten)         (None, 2048)              0         
                                                                 
 dense_12 (Dense)            (None, 1024)              2098176   
                                                                 
 dropout_8 (Dropout)         (None, 1024)              0         
                                                                 
 dense_13 (Dense)            (None, 512)               524800    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_14 (Dense)            (None, 10)               

In [22]:
prediction = model.predict(img)
pred_class = np.argmax(prediction, 1)
print('ground truth: ' + str(label))
print('prediction: ' + str(prediction))
print('materiale ground truth: ' + material)

for key in possible_materials.keys():
    if possible_materials[key] == pred_class:
        print('materiale predetto: ' + key)


1/1 [==============================] - 3s 3s/step
ground truth: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
prediction: [[0.0060548  0.00245433 0.00745299 0.32220435 0.4934124  0.01780049
  0.01154362 0.13468681 0.00248768 0.00190249]]
materiale ground truth: leather
materiale predetto: leather
